# Digital Twin Data Acquisition

In this notebook, we will use Big Query and The Graph to obtain the signals required for the Rai Digital Twin. We will get state snapshots and calculate the absolute changes between time periods. 


## Mapping of specification states to data
The quantity state variables of the system are:
* ETH in collateral = collateral
* ETH in Uniswap = reserve1
* RAI in Uniswap = reserve0 
* RAI drawn from SAFEs = erc20CoinTotalSupply

The metric state variables of the system are:

* Market Price of RAI in ETH = marketPriceEth
* Market Price of RAI in USD = marketPriceUsd
* Market Price of ETH in USD = ETH Price (OSM)

The metric control variables of the system are:

* Redemption Price of RAI in USD = RedemptionPrice
* Redemption Price of RAI in ETH = RedemptionPriceinEth

In [234]:
# import libraries
import os
from google.cloud import bigquery
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import numpy as np
import json
import requests
# The Graph URL
url = 'https://api.thegraph.com/subgraphs/name/reflexer-labs/rai-mainnet'

%matplotlib inline

# constants
constant = 1000000000000000000

#defining creditionals
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/aclarkdata/Downloads/raidata-5e1723d42cf5.json'

# initializing agent
client = bigquery.Client()


In [235]:
# SQL query
sql = """
SELECT 
* 
FROM `blockchain-etl.ethereum_rai.ExternallyFundedOSM_event_UpdateResult`
ORDER By block_timestamp ASC

"""

eth_price_OSM = client.query(sql).to_dataframe()
eth_price_OSM['ETH Price (OSM)'] = eth_price_OSM['newMedian'].astype(float)/constant
# subset
eth_price_OSM = eth_price_OSM[['block_number','ETH Price (OSM)','block_timestamp']]

eth_price_OSM.head()

,block_number,ETH Price (OSM),block_timestamp
0,12371493,3349.780000,2021-05-05 02:23:19+00:00
1,12371765,3349.780000,2021-05-05 03:22:45+00:00
2,12372918,3297.580000,2021-05-05 07:34:45+00:00
3,12373045,3274.547289,2021-05-05 08:04:07+00:00
4,12373322,3342.258435,2021-05-05 09:03:55+00:00


In [236]:
eth_price_OSM.tail()

,block_number,ETH Price (OSM),block_timestamp
399,12558237,2758.219111,2021-06-03 00:07:20+00:00
400,12559283,2713.885575,2021-06-03 04:06:02+00:00
401,12560042,2734.112143,2021-06-03 06:55:57+00:00
402,12560908,2843.991760,2021-06-03 10:05:55+00:00
403,12561983,2879.060471,2021-06-03 14:05:54+00:00


In [239]:
# SQL query
sql = """
SELECT 
* 
FROM `blockchain-etl.ethereum_rai.OSM_event_UpdateResult`
WHERE block_number < 12371493
ORDER By block_timestamp ASC

"""

eth_price_OSM_old = client.query(sql).to_dataframe()
eth_price_OSM_old['ETH Price (OSM)'] = eth_price_OSM_old['newMedian'].astype(float)/constant
# subset
eth_price_OSM_old = eth_price_OSM_old[['block_number','ETH Price (OSM)','block_timestamp']]

eth_price_OSM_old.head()

,block_number,ETH Price (OSM),block_timestamp
0,11848351,1803.656430,2021-02-13 12:46:18+00:00
1,11860755,1803.656430,2021-02-15 10:16:16+00:00
2,11861008,1763.974936,2021-02-15 11:13:28+00:00
3,11861237,1763.974936,2021-02-15 12:07:36+00:00
4,11861502,1775.977923,2021-02-15 13:07:59+00:00


In [240]:
eth_price_OSM_old.tail()

,block_number,ETH Price (OSM),block_timestamp
1885,12370391,3390.623840,2021-05-04 22:09:45+00:00
1886,12370625,3347.555806,2021-05-04 23:10:04+00:00
1887,12370903,3301.810000,2021-05-05 00:10:50+00:00
1888,12371173,3264.914896,2021-05-05 01:09:41+00:00
1889,12371432,3334.690000,2021-05-05 02:10:02+00:00


In [241]:
eth_price = eth_price_OSM_old.append(eth_price_OSM)

In [242]:
eth_price

,block_number,ETH Price (OSM),block_timestamp
0,11848351,1803.656430,2021-02-13 12:46:18+00:00
1,11860755,1803.656430,2021-02-15 10:16:16+00:00
2,11861008,1763.974936,2021-02-15 11:13:28+00:00
3,11861237,1763.974936,2021-02-15 12:07:36+00:00
4,11861502,1775.977923,2021-02-15 13:07:59+00:00
...,...,...,...
399,12558237,2758.219111,2021-06-03 00:07:20+00:00
400,12559283,2713.885575,2021-06-03 04:06:02+00:00
401,12560042,2734.112143,2021-06-03 06:55:57+00:00
402,12560908,2843.991760,2021-06-03 10:05:55+00:00


In [243]:
blocknumbers = eth_price.block_number.values.tolist()


In [244]:
uniswap_url = 'https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2'
pair = []
for i in blocknumbers:
    query = '''
    {
      pairs(block: {number:%s}, where: {id: "0x8ae720a71622e824f576b4a8c03031066548a3b1"}){
        id,
        token0 {
          id
        },
        token1 {
          id
        }
        reserve0,
        reserve1,
        totalSupply,
        reserveETH,
        reserveUSD,
        token0Price,
        token1Price
      }
    }
    
    ''' % i
    r = requests.post(uniswap_url, json = {'query':query})
    s = json.loads(r.content)['data']['pairs']
    pair.append(s)
        
pairState = pd.DataFrame(pair)

pairState['block_number'] = blocknumbers
pairState['data'] = pairState[0]
del pairState[0]
pairState = pairState[1:]


pairState['EthInUniswap'] = pairState.data.apply(lambda x: x['reserve1'])
pairState['RaiInUniswap'] = pairState.data.apply(lambda x: x['reserve0'])

# subset 
pairState = pairState[['block_number','EthInUniswap','RaiInUniswap']]
pairState.tail()



,block_number,EthInUniswap,RaiInUniswap
2289,12558237,6636.795727550091262062,6001286.63189994171306483
2290,12559283,6636.121069354291875144,5997659.370089077510315129
2291,12560042,6500.477040803870907552,6120939.631249455978822544
2292,12560908,6462.921777892681096872,6156614.927373514558597474
2293,12561983,6541.524901272872043101,6083153.098197337706001842


In [253]:
test  = pd.DataFrame(pair)
test['block_number'] = blocknumbers
test['data'] = test[0]
del test[0]
test = test[1:]
test

,block_number,data
1,11860755,{'id': '0x8ae720a71622e824f576b4a8c03031066548...
2,11861008,{'id': '0x8ae720a71622e824f576b4a8c03031066548...
3,11861237,{'id': '0x8ae720a71622e824f576b4a8c03031066548...
4,11861502,{'id': '0x8ae720a71622e824f576b4a8c03031066548...
5,11861791,{'id': '0x8ae720a71622e824f576b4a8c03031066548...
...,...,...
2289,12558237,{'id': '0x8ae720a71622e824f576b4a8c03031066548...
2290,12559283,{'id': '0x8ae720a71622e824f576b4a8c03031066548...
2291,12560042,{'id': '0x8ae720a71622e824f576b4a8c03031066548...
2292,12560908,{'id': '0x8ae720a71622e824f576b4a8c03031066548...


In [262]:
test[test['block_number'] == 12345529]['data'].values[0]

{'id': '0x8ae720a71622e824f576b4a8c03031066548a3b1',
 'reserve0': '18908049.836669973479224348',
 'reserve1': '20412.543157392702619562',
 'reserveETH': '40825.08631478540523912400000000001',
 'reserveUSD': '115195549.3881934103316527484790104',
 'token0': {'id': '0x03ab458634910aad20ef5f1c8ee96f1d6ac54919'},
 'token0Price': '926.2956453234561924515887747803531',
 'token1': {'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2'},
 'token1Price': '0.001079568931419090007655332093230422',
 'totalSupply': '599054.284772156919258854'}

In [245]:
state = []
for i in blocknumbers:
    query = '''
    {
      systemState(block: {number:%s},id:"current") { 
        coinUniswapPair {
          label
          reserve0
          reserve1
          token0Price
          token1Price
          totalSupply
        }
        currentCoinMedianizerUpdate{
          value
        }
        currentRedemptionRate {
          eightHourlyRate
          annualizedRate
          hourlyRate
          createdAt
        }
        currentRedemptionPrice {
          value
        }
        erc20CoinTotalSupply
        globalDebt
        globalDebtCeiling
        safeCount,
        totalActiveSafeCount
        coinAddress
        wethAddress
        systemSurplus
        debtAvailableToSettle
        lastPeriodicUpdate
        createdAt
        createdAtBlock
      }
    }
    ''' % i
    r = requests.post(url, json = {'query':query})
    s = json.loads(r.content)['data']['systemState']
    state.append(s)
        
systemState = pd.DataFrame(state)

systemState['block_number'] = blocknumbers

systemState = systemState[1:]

systemState.head()


,coinAddress,coinUniswapPair,createdAt,createdAtBlock,currentCoinMedianizerUpdate,currentRedemptionPrice,currentRedemptionRate,debtAvailableToSettle,erc20CoinTotalSupply,globalDebt,globalDebtCeiling,lastPeriodicUpdate,safeCount,systemSurplus,totalActiveSafeCount,wethAddress,block_number
1,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,None,1529678381,5834717,None,{'value': '3.14'},"{'annualizedRate': '1', 'createdAt': '16132260...",0,2788.440432971667032725,2788.522357955470745187436691184972,115792089237316195423570985008687.9,1613338681,3,0.06853325908830422221834559248594176,3,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,11860755
2,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,None,1529678381,5834717,None,{'value': '3.14'},"{'annualizedRate': '1', 'createdAt': '16132260...",0,2788.440432971667032725,2788.657935201718685876567866781341,115792089237316195423570985008687.9,1613384616,3,0.1087511150258265757085590152709335,3,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,11861008
3,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,None,1529678381,5834717,None,{'value': '3.14'},"{'annualizedRate': '1', 'createdAt': '16132260...",0,2788.440432971667032725,2788.657935201718685876567866781341,115792089237316195423570985008687.9,1613384616,3,0.1087511150258265757085590152709335,3,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,11861237
4,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,None,1529678381,5834717,None,{'value': '3.14'},"{'annualizedRate': '1', 'createdAt': '16132260...",0,3048.440432971667032725,3048.645954953510904626288422086589,115792089237316195423570985008687.9,1613393080,3,0.1161617985274885650688366678947734,3,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,11861502
5,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,None,1529678381,5834717,None,{'value': '3.14'},"{'annualizedRate': '1', 'createdAt': '16132260...",0,3048.440432971667032725,3048.645954953510904626288422086589,115792089237316195423570985008687.9,1613393080,3,0.1161617985274885650688366678947734,3,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,11861791


In [227]:
## reserve 0= rai
## reserve 1 = eth

# The quantity state variables of the system are:
# * ETH in collateral = collateral
# * ETH in Uniswap = reserve1
# * RAI in Uniswap = reserve0 
# * RAI drawn from SAFEs = erc20CoinTotalSupply
systemState.coinUniswapPair.values[-100:-99][0]

In [228]:
systemState.coinUniswapPair.values[-1]

{'label': 'UNISWAP_POOL_COIN',
 'reserve0': '6083793.130837400449920021',
 'reserve1': '6547.620994437380003343',
 'token0Price': '929.160856440219922635310199379282',
 'token1Price': '0.00107623991375527527434885805978229',
 'totalSupply': '189849.870522733048903617'}

In [246]:
systemState['RedemptionRateAnnualizedRate'] = systemState.currentRedemptionRate.apply(lambda x: x['annualizedRate'])    
systemState['RedemptionRateHourlyRate'] = systemState.currentRedemptionRate.apply(lambda x: x['hourlyRate'])
systemState['RedemptionRateEightHourlyRate'] = systemState.currentRedemptionRate.apply(lambda x: x['eightHourlyRate'])
systemState['RedemptionPrice'] = systemState.currentRedemptionPrice.apply(lambda x: x['value'])
# systemState['EthInUniswap'] = systemState.coinUniswapPair.apply(lambda x: x['reserve1'])
# systemState['RaiInUniswap'] = systemState.coinUniswapPair.apply(lambda x: x['reserve0'])
systemState['RaiDrawnFromSAFEs'] = systemState['erc20CoinTotalSupply']
#systemState['RAIInUniswapV2(RAI/ETH)'] = systemState.coinUniswapPair.apply(lambda x: x['reserve0'])
del systemState['currentRedemptionRate']
del systemState['currentRedemptionPrice']


# subset

systemState = systemState[['debtAvailableToSettle','globalDebt',
                           'globalDebtCeiling','systemSurplus','totalActiveSafeCount',
                           'block_number', 'RedemptionRateAnnualizedRate',
                           'RedemptionRateHourlyRate', 'RedemptionRateEightHourlyRate',
                        'RedemptionPrice','RaiDrawnFromSAFEs']]

In [247]:
hourly = []
for i in blocknumbers:
    query = '''
    {
      hourlyStats(where: {blockNumber_gt: %s}) { 
        marketPriceUsd # price of COIN in USD (uni pool price * ETH median price)
        marketPriceEth # Price of COIN in ETH (uni pool price)
      }
    }
    ''' % i
    r = requests.post(url, json = {'query':query})
    s = json.loads(r.content)['data']['hourlyStats'][0]
    hourly.append(s)
        
hourlyStats = pd.DataFrame(hourly)

hourlyStats['block_number'] = blocknumbers

hourlyStats.head()


,marketPriceEth,marketPriceUsd,block_number
0,0.002589462210048543650949087456484599,4.709364363675082915930020079963684,11848351
1,0.002481355746535171105569862943787003,4.362594137056344705089877147817243,11860755
2,0.002481355746535171105569862943787003,4.419265710853707004204445293455575,11861008
3,0.002481355746535171105569862943787003,4.419265710853707004204445293455575,11861237
4,0.001768934955701819449591545208406114,3.215446177235759811208395560891868,11861502


In [265]:
safehistories = []
for i in blocknumbers[1:]:
    query = '''
    {
      safes(block: {number:%s}) {
            collateral
            debt
      }
    }
    ''' % i
    r = requests.post(url, json = {'query':query})
    s = json.loads(r.content)['data']['safes']
    t = pd.DataFrame(s)
    t['collateral'] = t['collateral'].astype(float)
    t['debt'] = t['debt'].astype(float)
    safehistories.append(pd.DataFrame(t.sum().to_dict(),index=[0]))

safe_history = pd.concat(safehistories)


In [266]:
safe_history.head()

,collateral,debt
0,14.89236,2788.385291
0,14.89236,2788.385291
0,14.89236,2788.385291
0,16.89236,3048.358490
0,16.89236,3048.358490


In [267]:
safe_history['block_number'] = blocknumbers[1:]
safe_history.reset_index(inplace=True)
del safe_history['index']

In [268]:
states = hourlyStats.merge(systemState,how='inner',on='block_number')
states = states.merge(safe_history,how='inner',on='block_number')

In [269]:
states = states.merge(eth_price,how='inner',on='block_number')

In [270]:
states.head()

,marketPriceEth,marketPriceUsd,block_number,debtAvailableToSettle,globalDebt,globalDebtCeiling,systemSurplus,totalActiveSafeCount,RedemptionRateAnnualizedRate,RedemptionRateHourlyRate,RedemptionRateEightHourlyRate,RedemptionPrice,RaiDrawnFromSAFEs,collateral,debt,ETH Price (OSM),block_timestamp
0,0.002481355746535171105569862943787003,4.362594137056344705089877147817243,11860755,0,2788.522357955470745187436691184972,115792089237316195423570985008687.9,0.06853325908830422221834559248594176,3,1,1,1,3.14,2788.440432971667032725,14.89236,2788.385291,1803.656430,2021-02-15 10:16:16+00:00
1,0.002481355746535171105569862943787003,4.419265710853707004204445293455575,11861008,0,2788.657935201718685876567866781341,115792089237316195423570985008687.9,0.1087511150258265757085590152709335,3,1,1,1,3.14,2788.440432971667032725,14.89236,2788.385291,1763.974936,2021-02-15 11:13:28+00:00
2,0.002481355746535171105569862943787003,4.419265710853707004204445293455575,11861237,0,2788.657935201718685876567866781341,115792089237316195423570985008687.9,0.1087511150258265757085590152709335,3,1,1,1,3.14,2788.440432971667032725,14.89236,2788.385291,1763.974936,2021-02-15 12:07:36+00:00
3,0.001768934955701819449591545208406114,3.215446177235759811208395560891868,11861502,0,3048.645954953510904626288422086589,115792089237316195423570985008687.9,0.1161617985274885650688366678947734,3,1,1,1,3.14,3048.440432971667032725,16.89236,3048.358490,1775.977923,2021-02-15 13:07:59+00:00
4,0.001768934955701819449591545208406114,3.215446177235759811208395560891868,11861791,0,3048.645954953510904626288422086589,115792089237316195423570985008687.9,0.1161617985274885650688366678947734,3,1,1,1,3.14,3048.440432971667032725,16.89236,3048.358490,1805.792735,2021-02-15 14:07:59+00:00


In [271]:
states.tail(20)

,marketPriceEth,marketPriceUsd,block_number,debtAvailableToSettle,globalDebt,globalDebtCeiling,systemSurplus,totalActiveSafeCount,RedemptionRateAnnualizedRate,RedemptionRateHourlyRate,RedemptionRateEightHourlyRate,RedemptionPrice,RaiDrawnFromSAFEs,collateral,debt,ETH Price (OSM),block_timestamp
2273,0.001134960875267094467003791969743104,2.986696685159168706493653263262764,12542899,10898.18399270107882207376366334743,11515391.49509862110760467874908663,115792089237316195423570985008687.9,332732.8201321804329923361768877421,410,1.054683514310248,1.0000060777279998,1.000048622858297,3.007405427001464470882317416,11084931.026952268560592011,1348.804577,274011.310347,2532.678653,2021-05-31 15:25:55+00:00
2274,0.001143519253393445034445038085169669,3.019668422051002433558323170745842,12543889,10898.18399270107882207376366334743,11518263.74839866172399154363056714,115792089237316195423570985008687.9,332758.3185242162278939147897337467,411,0.9736160320565008,0.9999969476906555,0.9999755817861067,3.007400608487897605173380822,11085731.026952268560592011,1348.804577,274011.310347,2609.002086,2021-05-31 19:06:56+00:00
2275,0.001106268067735495297462765816839497,3.006426700551342260120562311245523,12544994,10898.18399270107882207376366334743,11530816.1375056087068609196539215,115792089237316195423570985008687.9,332816.5459191152795762975883410776,411,0.9736160320565008,0.9999969476906555,0.9999755817861067,3.007366932545723752615052643,11098191.340769549813994432,1348.804577,274011.310347,2599.093889,2021-05-31 23:05:58+00:00
2276,0.001134371874376024117133150291248161,2.987322956294284471505465965990382,12546090,10898.18399270107882207376366334743,11531540.23262161651002385946816772,115792089237316195423570985008687.9,332865.8300308467973695777222603854,411,0.8915770467022476,0.9999868992385849,0.9998951987141912,3.007250850648546312699156783,11098791.340769549813994432,1348.804577,274011.310347,2651.644184,2021-06-01 03:06:04+00:00
2277,0.001126189220841763318794341750295802,3.00905371727489899385341377919785,12547122,10898.18399270107882207376366334743,11544081.2811631428220325008100357,115792089237316195423570985008687.9,332937.4443086999517544485586625312,410,0.8915770467022476,0.9999868992385849,0.9998951987141912,3.007073873366830307101864472,11111186.838733311471356929,1348.804577,286935.660907,2637.017267,2021-06-01 07:05:48+00:00
2278,0.001162271091559327285847153961111834,2.978007371528899128755869640982682,12547598,10898.18399270107882207376366334743,11538661.21955852157440154749165619,115792089237316195423570985008687.9,332941.2881212844179001055790580126,410,0.9562468213913025,0.9999948927963887,0.9999591431014401,3.00702709676559652804905327,11105756.901462845024335037,1348.804577,281712.022468,2671.890000,2021-06-01 08:55:49+00:00
2279,0.001155336250318376509047419166714351,2.980282132311406520743333876532685,12548454,10898.18399270107882207376366334743,11539624.13684146079526976920462652,115792089237316195423570985008687.9,332975.8820226315420935301273368439,411,0.9562468213913025,0.9999948927963887,0.9999591431014401,3.006980990370253360930904171,11106656.901462845024335037,1348.804577,281712.022468,2577.566831,2021-06-01 12:05:54+00:00
2280,0.001164314592498290637544542511257835,2.985635507832059012510276365272665,12549522,10898.18399270107882207376366334743,11508166.33350450889002404220462652,115792089237316195423570985008687.9,332975.8820226315420935301273368439,409,0.9562468213913025,0.9999948927963887,0.9999591431014401,3.006914289216020773899974605,11075014.616684485311160477,1348.804577,281712.022468,2609.774303,2021-06-01 16:05:38+00:00
2281,0.001171140507100952541172318741318186,3.001792549492945270464064007387189,12550684,10898.18399270107882207376366334743,11523064.16689370140140164891182405,115792089237316195423570985008687.9,333058.2493147244334644767620019062,409,1.0339608700071055,1.0000038124424329,1.0000304999464383,3.006952450432485284713018857,11090014.616684485311160477,1348.804577,296624.355518,2564

In [273]:
# export
states.to_csv('states.csv')